In [ ]:
import xlrd
import csv
import pickle
import numpy as np
import copy
from matplotlib import pyplot as plt
import math
import random
import pandas as pd
from numpy.random import choice
from scipy import stats
from collections import defaultdict
from operator import itemgetter, sub
from sklearn.metrics import roc_curve, auc, accuracy_score, explained_variance_score, \
    mean_squared_error, mean_absolute_error, median_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, ElasticNet, Lasso
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [ ]:
def getSamplestopkl(filename):
        'convert datasSelected to Dataframe\
         --row index begins at 1\
         -- only cell with five neighbors now'
        data = pd.read_csv(filename)
        dataSelected = data.loc[:,['Iteration Num','CellId','PilotPower','Load','Users','Cluster','TBS(kbit)']]
 
        dataLen = len(dataSelected.index)
        df = pd.DataFrame.from_records([29*[None]],columns=['Iteration','CellId','PilotPower','Load','Users','TBS',\
                                                         'NB1Id','NB2Id','NB3Id','NB4Id','NB5Id',\
                                                         'NB1Power','NB2Power','NB3Power','NB4Power','NB5Power',\
                                                         'NB1Users','NB2Users','NB3Users','NB4Users','NB5Users',\
                                                         'NB1Load','NB2Load','NB3Load','NB4Load','NB5Load',\
                                                         'AveragePower','AverageLoad','AverageUser'])
        for numIter in range(dataLen):    
            curStr = dataSelected.loc[numIter,'Cluster']
            curStrList = curStr.split(';')
            # if number of neighbors <6
            if len(curStrList)<6:
                        continue 
            #delete last item ''
            curStrList=curStrList[0:5]                  
            #features:Iteration','CellId','PilotPower','Load','Users','TBS'
            curList = []
            curList += [float(dataSelected.loc[numIter,'Iteration Num'])]         
            curList += [float(dataSelected.loc[numIter,'CellId'])]                
            curList += [float(dataSelected.loc[numIter,'PilotPower'])]            
            curList += [float(dataSelected.loc[numIter,'Load'])]                  
            curList += [float(dataSelected.loc[numIter,'Users'])]                 
            curList += [float(dataSelected.loc[numIter,'TBS(kbit)'])]             

            neighborCellId= [None] * 5
            curNeiborPower=[]
            curNeiborUsers=[]
            curNeiborLoad=[]
            average= [0] * 3
            
            #features:neighbor's 'CellId','PilotPower','Load','Users'
            curDataIter = dataSelected.loc[numIter,'Iteration Num']
            for i in range(5):
                neighborCellId[i] = int(curStrList[i])
                neighbor = dataSelected[(dataSelected['Iteration Num']== curDataIter) \
                                        & (dataSelected['CellId']==neighborCellId[i])]
                curNeiborPower += [float(neighbor.loc[:,'PilotPower'])]
                curNeiborLoad += [float(neighbor.loc[:,'Load'])]
                curNeiborUsers += [float(neighbor.loc[:,'Users'])]
                
            #features: average of neighbor's PilotPower','Load','Users'
            for i in range(5):
                average[0] += curNeiborPower[i]/5                        
                average[1] +=  curNeiborLoad[i]/5                         
                average[2] += curNeiborUsers[i]/5 
                
            curList=curList+neighborCellId+curNeiborPower+curNeiborUsers+curNeiborLoad+average
            curList=[curList]
            CurItem = pd.DataFrame.from_records(curList, columns=['Iteration','CellId','PilotPower','Load','Users',\
                                                                  'TBS','NB1Id','NB2Id','NB3Id','NB4Id','NB5Id',\
                                                              'NB1Power','NB2Power','NB3Power','NB4Power','NB5Power',\
                                                              'NB1Users','NB2Users','NB3Users','NB4Users','NB5Users',\
                                                         'NB1Load','NB2Load','NB3Load','NB4Load','NB5Load',\
                                                         'AveragePower','AverageLoad','AverageUser'])
            # add the row to the dataframe
            df=df.append(CurItem, ignore_index=True)
            
        # delete the row(index=0)
        df=df.drop(df.index[[0]])

        # Add features:TimeAveragePower,TimeAverageLoad,TimeAverageUsers to DataFrame
        TimeAveragePower = []
        TimeAverageLoad = []
        TimeAverageUsers = []
        for i in df.index:
            CellIndex = float(df.loc[i, ['CellId']])
            TimeAveragePower += [df[df.CellId == CellIndex]['PilotPower'].mean()]
            TimeAverageLoad += [df[df.CellId == CellIndex]['Load'].mean()]
            TimeAverageUsers += [df[df.CellId == CellIndex]['Users'].mean()]
        df['TimeAveragePower'] = TimeAveragePower
        df['TimeAverageLoad'] = TimeAverageLoad
        df['TimeAverageUsers'] = TimeAverageUsers  
        
        df.to_pickle('Data__'+filename+'.pkl')  
        